In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy_ext as npext
import pandas as pd
import pyximport

pyximport.install(language_level=3,
                  setup_args={'include_dirs': [np.get_include(), npext.np.get_include()]})
from matplotlib import cm
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, SpectralEmbedding
from sklearn.preprocessing import MinMaxScaler
import model
from model import CalculateFeatures, SelectFeatures
%matplotlib qt


In [2]:
pdf = pd.DataFrame
df = model.load_data('../../data/DATA_3_south.csv')
df.head()

,Magnitude,Latitude,Longitude,Depth,Datetime
0,3.9,16.8023,-94.9130,114.9,2017-01-01 00:54:31
1,3.9,15.2842,-93.1210,100.7,2017-01-01 04:10:39
2,3.9,17.4503,-94.9543,138.3,2017-01-01 04:23:51
3,3.9,16.4077,-93.7075,139.4,2017-01-01 04:26:01
4,4.1,16.7017,-98.8633,27.5,2017-01-01 08:49:23


In [3]:
df_model = CalculateFeatures(df, 50, trim_features=True)

In [4]:
features = df_model.features
features.head()

,old_index,firstT,lastT,meanMag,maxAMag,maxEMag,magDef,a,b,bStd,grcStd,elapsedT,rateSqrtEnergy,meanT,meanTStd,pMag,zSeismicRateChange,bSeismicRateChange,lastDMaxMag,nextDMaxMag
0,72,2017-01-02 04:22:50,2017-01-08 00:57:59,3.968,4.8,4.621209,0.178791,9.59308,2.075881,0.234991,42.276599,506109,80703.020164,10328.755102,1.483613e+09,3.505225e-13,180.138835,-6.383146,4.8,5.0
1,73,2017-01-02 06:34:09,2017-01-08 11:24:47,3.964,4.8,4.603285,0.196715,9.75370,2.118856,0.246455,42.326895,535838,75792.567758,10935.469388,1.483623e+09,1.935804e-13,1219.200558,292.410678,4.8,5.0
2,74,2017-01-02 07:16:36,2017-01-08 12:16:03,3.978,4.8,4.661154,0.138846,9.19042,1.971705,0.236863,24.754354,536367,78645.619778,10946.265306,1.483634e+09,1.478329e-12,162.634560,5.059640,4.8,5.0
3,75,2017-01-02 08:31:38,2017-01-08 21:15:56,3.978,4.8,4.661154,0.138846,9.19042,1.971705,0.236863,24.754354,564258,74758.204835,11515.469388,1.483644e+09,1.478329e-12,1180.910666,266.499676,4.8,5.0
4,76,2017-01-02 14:48:51,2017-01-08 22:53:08,3.982,4.8,4.678352,0.121648,9.05955,1.936483,0.226983,24.719963,547457,77476.225627,11172.591837,1.483656e+09,2.404915e-12,916.542416,-156.641921,4.8,5.0


In [8]:
df_model.features.to_csv("../../model/features_cache/features.csv", index=False)

In [9]:
features.sort_values('nextDMaxMag',inplace=True)

In [10]:
X = features[model.FEATURES]
Y = features[model.TARGETS]
selected_features = SelectFeatures(features=X, targets=Y, corr_threshold=.95, fixed_features=["b"])

X = X[selected_features[0]].to_numpy()
Y = Y.to_numpy()

In [11]:
Y_Scaler = MinMaxScaler()
X_Scaler = MinMaxScaler()
X_norm64 = X_Scaler.fit_transform(X)
Y_norm64 = Y_Scaler.fit_transform(Y)
X_norm32 = X_norm64.astype(np.float32)
Y_norm32 = Y_norm64.astype(np.float32)

In [12]:
# evaluate isomap with logistic regression algorithm for classification
pca = PCA(n_components=2)
pca.fit(X_norm64.T, Y_norm64.T)
pca_components = pca.components_

In [13]:
cmap = cm.get_cmap("viridis")
colors = list(map(cmap, Y_norm64.T[0]))
sizes = np.power(10, Y_norm64.T[0])

In [14]:
%matplotlib qt
plt.scatter(pca_components[0], pca_components[1], s=sizes, color=colors)

In [31]:
import seaborn as sn
import matplotlib.pyplot as plt

corrMatrix = df_model.features.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()


In [39]:
embedding = Isomap(n_components=2)
X_transformed = embedding.fit_transform(X_norm32, Y_norm32)

%matplotlib qt
fix, ax = plt.subplots()
ax.set_facecolor((0, 0, .420, 0.125))
ax.scatter(X_transformed.T[0], X_transformed.T[1], s=sizes, color=colors)

In [183]:
import matplotlib.pyplot as plt
% matplotlib qt

fig, ax = plt.subplots()
num_ticks = np.round((Y_Scaler.data_max_ - Y_Scaler.data_min_) * 10 + 1).astype(int)[0]
labels = np.round(np.linspace(Y_Scaler.data_min_, Y_Scaler.data_max_, num=num_ticks).T[0], 1)
ticks = np.linspace(0, 1, num=num_ticks)[np.isin(labels, np.unique(Y))]
cax = plt.scatter(X_transformed.T[0], X_transformed.T[1], s=sizes, color=colors)

cbar = fig.colorbar(cax, ticks=ticks)
cbar.ax.set_yticklabels(np.unique(Y))
plt.plot()

[]

In [ ]:
% matplotlib qt
import seaborn as sns
import matplotlib.pyplot as plt

corrMatrix = features.corr()

sns.heatmap(corrMatrix, annot=True, cmap="bwr")
plt.show()


In [191]:
Embedding = SpectralEmbedding(n_components=2)
Embedding.fit(X_norm32, Y_norm32)

SystemError: gstrf was called with invalid arguments